# test icewebSpectrogram

Test the Python version of IceWeb. Under development.

Last update: Glenn Thompson 2021/06/21

In [1]:
""" 
We are working here from the WAV files in Seisan. 
We ideally need a process that checks for REA files that exist without corresponding WAV files too.

Plus we might want to build a version of this from an Antelope catalog.
"""

import os, sys
import numpy as np
from obspy import read, read_inventory, Stream
from obspy.io.xseed.core import _read_resp
#from obspy.signal.freqattributes import spectrum
from obspy.imaging.cm import obspy_sequential


sys.path.append('/Users/thompsong/src/kitchensinkGT/LIB')
#import libseisGT
from libMVO import correct_nslc
from metrics import process_trace

sys.path.append('/Users/thompsong/src/icewebPy')
import IceWeb

            
def seisandb2miniseed(DB, YYYY, MM, shortperiod):
    eventdir = os.path.join('WAV', DB, YYYY, MM)
    badseisanfiles = []
    pngdir = eventdir.replace('WAV', 'MSEED')
    print(eventdir)
    print(pngdir)
    if not os.path.exists(pngdir):
        os.makedirs(pngdir)
    for root, dirs, files in os.walk(eventdir, topdown=False):
        files = sorted(files)
        files = files[0:4]
        for seisanfile in files:
            print('Processing %s' % seisanfile)
            st = Stream()
            
            # check for non-seisan files
            if seisanfile[-4:]=='.png':
                #os.remove(thisfile)
                continue
            if not (seisanfile[0:2]=='19' or seisanfile[0:2]=='20'):
                print('Unrecognized file: %s' % seisanfile)
                continue
                
            seisanfullpath = os.path.join(root, seisanfile)    
            rawpngfile = os.path.join(pngdir, seisanfile + '_raw.png')
            correctedpngfile = rawpngfile.replace('_raw', '_corrected')
            rawmseedfile = rawpngfile.replace('.png', '.mseed')
            correctedmseedfile = correctedpngfile.replace('.png', '.mseed')
            picklefile = rawmseedfile.replace('.mseed', '.pickle')
            
            if os.path.exists(rawmseedfile):
                #pass
                os.remove(rawmseedfile)
            
            if not os.path.exists(rawmseedfile):
                try:
                    print('- reading %s' % seisanfile)
                    st = read(seisanfullpath)        
                except:
                    badseisanfiles.append(seisanfile)
            
            #print(st)
            if len(st)==0:
                print('No traces loaded')
                continue

            # write raw Miniseed file and create plot of Z-components only
            print('- writing %s' % rawmseedfile)
            st.write(rawmseedfile)
            st.plot(equal_scale=False, outfile=rawpngfile, dpi=100); 

            for tr in st:
                                    
                ## find inventory
                this_inv = None
                tr.stats.network = 'MN' # temporary, to match RESP files, should be 'MV'
                respfile = os.path.join('CAL', "RESP.%s" % tr.id)
                xmlfile = os.path.join(pngdir, "inventory.%s.xml" % tr.id)
                #print('StationXML file = ',xmlfile)
                if os.path.exists(xmlfile):
                    this_inv = read_inventory(xmlfile)
                elif os.path.exists(respfile):
                    print('RESP file = ',respfile)
                    this_inv = _read_resp(respfile)
                    this_inv.write(xmlfile , format="STATIONXML")
                #print(this_inv)
                
                # data QC and clean (detrend, filter and correct) trace
                process_trace(tr, inv=this_inv)
                if tr.stats.quality_factor == 0:
                    st.remove(tr)
                    continue
                
                # fix trace ID
                tr.id = correct_nslc(tr.id, tr.stats.sampling_rate, shortperiod=shortperiod)
                
                #print(tr.stats)
                print('-- %s, trace quality = %d' % (tr.id, tr.stats.quality_factor))

            # WRITE A PICKLE FILE HERE
            st.write(picklefile, format='PICKLE')
            
            stZ = st.select(component='Z')
            stZ.plot(equal_scale=True, outfile=correctedpngfile, dpi=100);
    return badseisanfiles




def process_miniseed_files(DB, YYYY, MM):                       

    eventdir = os.path.join('MSEED', DB, YYYY, MM)
    
    for root, dirs, files in os.walk(eventdir, topdown=False):
        files = sorted(files)
        files = files[0:4]
        for file in files:
            if not '_corrected.mseed' in file:
                continue
                
            print('Processing %s' % file)
            mseedfullpath = os.path.join(root, file) 
            st = read(mseedfullpath)
            print(st)
            stZ = st.select(component='Z')    
      
            # spectrogram
            iwsobj = IceWeb.icewebSpectrogram(stream=stZ)
            sgramfile = mseedfullpath.replace('_corrected.mseed', '_sgram.png')
            iwsobj = iwsobj.precompute()
            iwsobj.plot(outfile=sgramfile, log=False, equal_scale=True, add_colorbar=True, dbscale=True, title="%s" % os.path.basename(correctedmseedfile))
            # could compute for all channels and then add a component parameter to limit channels plotted
            
            # measure data quality & compute metrics
            
            
            # build web page
            htmlfile = file.replace('_corrected.mseed', '.html')
            build_event_webpage(htmlfile, streamplot=file, sgramplot=sgramfile, metricsdf = metricsdf)
            
            
            # ampengfft - need to figure best model for this out
            #iwsobj.ampengfft() 
                       
            continue 
            """
            # add an ampengfft method to IceWeb.icewebSpectrogram and modify iwsobj.stream
            # but rather than just replicate old ampengfft which was more like ampengssam,
            # probably better to return full FFT, just as in frequency metrics sampled once-per-minute in GISMO/+iceweb
            # The easy way to do it for events or 10-min or 20-min continuous data chunks is probably:
            1. iwsobj = IceWeb.icewebSpectrogram(st)
            2. iwsobj = iwsobj.precompute()
            3. aef = ampengfft(iwsobj)
            aef here would be a list of dictionaries, one per Trace
            each dict would contain maxamp, energy, ampspec
            do we want to break these down for each minute, for both event and continuous data?
            
            let's also add method for instrument corrected RSAM from my course or the White Island project
            check Miami Lakes too
            Do I also want to archive SSAM?
            """          
            

            if not aef:
                continue
        
            ampengfft(iwsobj)
        
            # write pickle file
            picklefile = mseedfullpath.replace('_corrected.mseed', '.pickle')
            st.write(picklefile, format='PICKLE')
        
            # write an AEF-file
            fptr = open(aeffile, 'w')
            fptr.write('NSLC, amp, eng, f0-1, f1-2, f2-3, f3-4, f4-5, f5-6, f6-7, f7-8, f8-9, f9-10, f10-11, f11-12, f12-13, f13-14, f14-15, f15-16\n')
            for tr in st:
                fptr.write('%s, %4.2e, %4.2e' % (tr.id, tr.stats.peakamp, tr.stats.energy))
                if 'ssam' in tr.stats:
                    for x in tr.stats.ssam:
                        fptr.write(', %4.2e' % (x/tr.stats.peakamp))
                fptr.write('\n')
            fptr.close()

def build_event_webpage(htmlfile, streamplot=None, sgramplot=None, metricsdf = None):
    pass
        
# Montserrat Seisan data processor, one month at a time
os.chdir('%s/DATA/MVO' % os.getenv('HOME'))
DB = 'MVOE_'
YYYY = '2005'
MM = '05'
shortperiod = False
if DB[0:4]=='ASNE':
    shortperiod = True
badseisanfiles = seisandb2miniseed(DB, YYYY, MM, shortperiod)
print('\nBad Seisan files:')
print(badseisanfiles)
print("\n")
#process_miniseed_files(DB, YYYY, MM)

WAV/MVOE_/2005/05
MSEED/MVOE_/2005/05
Processing 2005-05-01-0149-37S.MVO___025
- reading 2005-05-01-0149-37S.MVO___025
- writing MSEED/MVOE_/2005/05/2005-05-01-0149-37S.MVO___025_raw.mseed
Outliers: 0
(100, 91)
(91,)
-- MV.MBBY..HHZ, trace quality = 7
Outliers: 0
(100, 91)
(91,)
-- MV.MBBY..HHN, trace quality = 7
Outliers: 0
(100, 91)
(91,)
-- MV.MBBY..HHE, trace quality = 7
Outliers: 0
(100, 91)
(91,)
-- MV.MBGB..HHZ, trace quality = 7
Outliers: 0
(100, 91)
(91,)
-- MV.MBGB..HHN, trace quality = 7
Outliers: 0
(100, 91)
(91,)
-- MV.MBGB..HHE, trace quality = 7
Outliers: 0
(100, 91)
(91,)
-- MV.MBGH..HHZ, trace quality = 7
Outliers: 0
(100, 91)
(91,)
-- MV.MBGH..HHN, trace quality = 7
Outliers: 0
(100, 91)
(91,)
-- MV.MBGH..HHE, trace quality = 7
Outliers: 0
(100, 91)
(91,)
-- MV.MBHA..HHZ, trace quality = 5
Outliers: 0
(100, 91)
(91,)
-- MV.MBHA..HHN, trace quality = 7
Outliers: 0
(100, 91)
(91,)
-- MV.MBHA..HHE, trace quality = 7
Outliers: 0
(100, 91)
(91,)
-- MV.MBLY..HHZ, trace qual